In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.datasets import mnist
from tensorflow.contrib.layers import batch_norm, fully_connected


def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [22]:

reset_graph()
'''Preparing===================================================================='''
(X_train, y_train), (X_test, y_test) = mnist.load_data()
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

n_train = 60000
n_test = 10000

X_train1 = X_train[:n_train]
y_train1 = y_train[:n_train].astype(np.int32)
X_test1 = X_test[:n_test]
y_test1 = y_test[:n_test].astype(np.int32)

'''Using SVD to rotate image===================================================='''
sigma = 1
def svd(all_images):
    new_all_images = []
    for image in all_images:
        U, S, VT = np.linalg.svd(image, full_matrices=0)
        N = 28
        cutoff = (4 / np.sqrt(3)) * np.sqrt(N) * sigma
        r = np.max(np.where(S > cutoff))
        
        new_image = U[:, :(r+1)] @ np.diag(S[ :(r+1)]) @ VT[ :(r+1), :]
        new_all_images.append(new_image)
    return new_all_images
'''============================================================================='''

X_train = np.array(svd(X_train1)).astype(np.float32) / 255.0
X_test = np.array(svd(X_test1)).astype(np.float32) / 255.0

X_train = X_train.reshape(n_train, 28*28)
X_test = X_test.reshape(n_test, 28*28)
y_train = y_train1
y_test = y_test1

In [ ]:
m_train, n_train = X_train.shape
print(X_train.shape)

n_epochs = 11
batch_size = 50
n_batches = int(np.ceil(m_train/batch_size))

def fetch_batch(epoch, batch_index):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m_train, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_train, y_train

'''placeholder==============================================='''
X = tf.placeholder(dtype=tf.float32, shape=(None, 28*28), name='X')
y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')


is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
bn_params = {
 'is_training': is_training,
 'decay': 0.99,
 'updates_collections': None
}

'''hidden layer=================================================================================================='''
hidden1 = fully_connected(X, n_hidden1, scope="hidden1", normalizer_fn=batch_norm, normalizer_params=bn_params)

hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2", normalizer_fn=batch_norm, normalizer_params=bn_params)

logits = fully_connected(hidden2, n_outputs, activation_fn=None,scope="outputs", normalizer_fn=batch_norm, normalizer_params=bn_params)

'''Loss======================================='''
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name='loss')

'''optimizer==============================================================='''
def learning_schedule(t):
    return 1 / (100 * (t+1))

t = tf.placeholder(dtype=tf.float32, shape=(None))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_schedule(t))
training_op = optimizer.minimize(loss)

'''accuracy======================================================================================'''
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index)
            _accuracy, _loss, _ = sess.run([accuracy, loss, training_op], feed_dict={X: X_batch, y: y_batch, t: epoch})
        _test_accuracy = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
        if (epoch % 5 == 0):
          print("\t\tEpoch: {} \ttLoss: {}, \tTrain_Accuracy: {} \tTest_Accuracy: {}".format(epoch, _loss, _accuracy*100, _test_accuracy*100))

(60000, 784)
		Epoch: 0 	tLoss: 1.9295901438454166e-05, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.58999943733215


# Test-2: Don't use SVD

In [21]:
from tensorflow.keras.datasets import mnist
from tensorflow.contrib.layers import batch_norm, fully_connected
reset_graph()
'''Preparing==============================================='''
(X_train, y_train), (X_test, y_test) = mnist.load_data()
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X_train = X_train[:3000].reshape(len(X_train[:3000]), 28*28).astype(np.float32)/255.0
y_train = y_train[:3000].astype(np.int32)
X_test = X_test[:1000].reshape(len(X_test[:1000]), 28*28).astype(np.float32)/255.0
y_test = y_test[:1000].astype(np.int32)

# def one_hot(inputs):
#     one_hot_encoded = np.zeros((len(inputs), 10))
#     for i, j in enumerate(inputs):
#         one_hot_encoded[i][j] = 1
#     return one_hot_encoded

# y_train = one_hot(y_train)
# y_test = one_hot(y_test)

m_train, n_train = X_train.shape
print(X_train.shape)

n_epochs = 101
batch_size = 50
n_batches = int(np.ceil(m_train/batch_size))

def fetch_batch(epoch, batch_index):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m_train, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_train, y_train

'''placeholder==============================================='''
X = tf.placeholder(dtype=tf.float32, shape=(None, 28*28), name='X')
y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')


is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
bn_params = {
 'is_training': is_training,
 'decay': 0.99,
 'updates_collections': None
}

'''hidden layer=================================================================================================='''
hidden1 = fully_connected(X, n_hidden1, scope="hidden1", normalizer_fn=batch_norm, normalizer_params=bn_params)

hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2", normalizer_fn=batch_norm, normalizer_params=bn_params)

logits = fully_connected(hidden2, n_outputs, activation_fn=None,scope="outputs", normalizer_fn=batch_norm, normalizer_params=bn_params)

'''Loss======================================='''
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name='loss')

'''optimizer==============================================================='''
def learning_schedule(t):
    return 1 / (100 * (t+1))

t = tf.placeholder(dtype=tf.float32, shape=(None))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_schedule(t))
training_op = optimizer.minimize(loss)

'''accuracy======================================================================================'''
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index)
            _accuracy, _loss, _ = sess.run([accuracy, loss, training_op], feed_dict={X: X_batch, y: y_batch, t: epoch})
        _test_accuracy = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
        if (epoch % 50 == 0):
          print("\t\tEpoch: {} \ttLoss: {}, \tTrain_Accuracy: {} \tTest_Accuracy: {}".format(epoch, _loss, _accuracy*100, _test_accuracy*100))

(3000, 784)
		Epoch: 0 	tLoss: 0.005409108940511942, 	Train_Accuracy: 99.9666690826416 	Test_Accuracy: 92.29999780654907
		Epoch: 50 	tLoss: 3.579313124646433e-05, 	Train_Accuracy: 100.0 	Test_Accuracy: 92.69999861717224
		Epoch: 100 	tLoss: 1.1595784599194303e-05, 	Train_Accuracy: 100.0 	Test_Accuracy: 92.79999732971191
